In [20]:
import pandas as pd
import numpy as np
import itertools

In [21]:
btc_prices = pd.read_csv('Bitcoin (USDT) 2024 timeframe 1m.csv')
btc_prices.set_index('timestamp', inplace=True)
btc_prices.index = pd.to_datetime(btc_prices.index)
btc_prices

,open,high,low,close,volume
timestamp,,,,,
2024-01-01 00:00:00,42284.00,42300.00,42260.01,42300.00,15.059749
2024-01-01 00:01:00,42300.00,42321.99,42299.99,42320.28,4.240722
2024-01-01 00:02:00,42320.28,42335.79,42317.30,42322.57,5.671511
2024-01-01 00:03:00,42322.57,42370.00,42322.57,42366.50,6.801620
2024-01-01 00:04:00,42366.50,42399.68,42366.50,42399.68,6.264730
...,...,...,...,...,...
2024-12-30 23:56:00,92816.59,92878.51,92789.85,92870.74,6.900125
2024-12-30 23:57:00,92870.74,92895.11,92838.44,92844.67,6.369311
2024-12-30 23:58:00,92844.67,92845.68,92797.10,92831.16,6.396956


In [22]:
def decide_action(timestamp, price):
    """
    Funzione che decide se aprire una posizione.
    Per ora restituisce sempre "long".
    """
    return "long"

In [23]:
def simulate_trading(btc_prices, TP, SL, L, p_investment, initial_liquidity, decision_function):
    """
    Simula la strategia di trading sul dataframe btc_prices con i parametri:
      - TP: take profit (es. 0.2 per +20%)
      - SL: stop loss (es. 0.1 per -10%)
      - L: leva (intero tra 1 e 100)
      - p_investment: percentuale della liquidità da investire in ogni trade
    La funzione restituisce un dataframe con:
      - timestamp, price, action, liquidity, btc_held, position_value, total_value.
    """
    current_liquidity = initial_liquidity  # liquidità corrente
    open_trade = None  # nessuna posizione aperta all'inizio
    output_rows = []
    
    # Iteriamo riga per riga (ogni riga rappresenta 1 minuto)
    for timestamp, row in btc_prices.iterrows():
        price = row['close']
        
        # Se non c'è posizione aperta, decidiamo se aprirne una
        if open_trade is None:
            action_decision = decision_function(timestamp, price)
            if action_decision in ["long", "short"]:
                if action_decision == "long":
                    # Apertura di una posizione long
                    margin = current_liquidity * p_investment        # margine investito
                    free_liquidity = current_liquidity - margin       # liquidità libera dopo aver bloccato il margine
                    notional = margin * L                             # valore nozionale della posizione
                    quantity = notional / price                       # quantità di BTC acquistati
                    entry_price = price
                    tp_price = entry_price * (1 + TP)
                    sl_price = entry_price * (1 - SL)
                    # Calcoliamo il prezzo di liquidazione: distanza percentuale = (100/L - 0.5)%
                    liq_threshold_pct = (100 / L) - 0.5
                    liq_price = entry_price * (1 - liq_threshold_pct / 100)
                    
                    open_trade = {
                        'direction': 'long',
                        'entry_price': entry_price,
                        'margin': margin,
                        'free_liquidity': free_liquidity,
                        'quantity': quantity,
                        'tp_price': tp_price,
                        'sl_price': sl_price,
                        'liq_price': liq_price
                    }
                    row_action = "open long"
                    
                    # Registra la riga con lo stato di apertura
                    total_value = free_liquidity + quantity * price
                    output_rows.append({
                        'timestamp': timestamp,
                        'price': price,
                        'action': row_action,
                        'liquidity': free_liquidity,
                        'btc_held': quantity,
                        'position_value': quantity * price,
                        'total_value': total_value
                    })
            else:
                # Nessuna posizione aperta e nessuna decisione di aprirla
                output_rows.append({
                    'timestamp': timestamp,
                    'price': price,
                    'action': "-",
                    'liquidity': current_liquidity,
                    'btc_held': 0,
                    'position_value': 0,
                    'total_value': current_liquidity
                })
        else:
            # Gestione della posizione aperta
            trade = open_trade
            direction = trade['direction']
            entry_price = trade['entry_price']
            quantity = trade['quantity']
            free_liquidity = trade['free_liquidity']
            
            if direction == 'long':
                # Condizione per take profit
                if price >= trade['tp_price']:
                    profit = quantity * (price - entry_price)
                    current_liquidity = free_liquidity + trade['margin'] + profit
                    output_rows.append({
                        'timestamp': timestamp,
                        'price': price,
                        'action': "close long (TP)",
                        'liquidity': current_liquidity,
                        'btc_held': 0,
                        'position_value': 0,
                        'total_value': current_liquidity
                    })
                    open_trade = None
                # Condizione per stop loss
                elif price <= trade['sl_price']:
                    loss = quantity * (price - entry_price)  # sarà negativo
                    current_liquidity = free_liquidity + trade['margin'] + loss
                    output_rows.append({
                        'timestamp': timestamp,
                        'price': price,
                        'action': "close long (SL)",
                        'liquidity': current_liquidity,
                        'btc_held': 0,
                        'position_value': 0,
                        'total_value': current_liquidity
                    })
                    open_trade = None
                # Condizione di liquidazione
                elif price <= trade['liq_price']:
                    # In liquidazione, si perde l'intero margine investito
                    current_liquidity = free_liquidity
                    output_rows.append({
                        'timestamp': timestamp,
                        'price': price,
                        'action': "close long (LIQ)",
                        'liquidity': current_liquidity,
                        'btc_held': 0,
                        'position_value': 0,
                        'total_value': current_liquidity
                    })
                    open_trade = None
                else:
                    # La posizione rimane aperta: si registra lo stato corrente
                    total_value = free_liquidity + quantity * price
                    output_rows.append({
                        'timestamp': timestamp,
                        'price': price,
                        'action': "long",
                        'liquidity': free_liquidity,
                        'btc_held': quantity,
                        'position_value': quantity * price,
                        'total_value': total_value
                    })
            else:
                # Per le posizioni short (non implementato in questo esempio)
                pass

    # Se alla fine del dataframe c'è ancora una posizione aperta, la chiudiamo al prezzo finale
    if open_trade is not None:
        trade = open_trade
        if trade['direction'] == 'long':
            profit = trade['quantity'] * (price - trade['entry_price'])
            current_liquidity = trade['free_liquidity'] + trade['margin'] + profit
            output_rows.append({
                'timestamp': timestamp,
                'price': price,
                'action': "close long (end)",
                'liquidity': current_liquidity,
                'btc_held': 0,
                'position_value': 0,
                'total_value': current_liquidity
            })
    
    df_sim = pd.DataFrame(output_rows)
    df_sim.set_index('timestamp', inplace=True)
    return df_sim

In [24]:
# Esempio di test:
df_simulation = simulate_trading(btc_prices, TP=0.2, SL=0.1, L=5, p_investment=0.1, initial_liquidity=1000, decision_function=decide_action)
df_simulation

,price,action,liquidity,btc_held,position_value,total_value
timestamp,,,,,,
2024-01-01 00:00:00,42300.00,open long,900.000000,0.011820,500.000000,1400.000000
2024-01-01 00:01:00,42320.28,long,900.000000,0.011820,500.239716,1400.239716
2024-01-01 00:02:00,42322.57,long,900.000000,0.011820,500.266785,1400.266785
2024-01-01 00:03:00,42366.50,long,900.000000,0.011820,500.786052,1400.786052
2024-01-01 00:04:00,42399.68,long,900.000000,0.011820,501.178251,1401.178251
...,...,...,...,...,...,...
2024-12-30 23:57:00,92844.67,long,1416.612156,0.008073,749.504743,2166.116900
2024-12-30 23:58:00,92831.16,long,1416.612156,0.008073,749.395681,2166.007838
2024-12-30 23:59:00,92789.06,long,1416.612156,0.008073,749.055822,2165.667978


In [26]:
df_simulation['action'].value_counts()

action
long                525572
open long               15
close long (TP)          8
close long (SL)          6
close long (end)         1
Name: count, dtype: int64

In [ ]:
# Cell 4: Grid Search sulle combinazioni di parametri con messaggi di progresso

initial_liquidity = 1000

# Definiamo i possibili valori per ciascun parametro
TP_values = [0.1, 0.2, 0.3]             # es. +10%, +20%, +30%
SL_values = [0.05, 0.1, 0.15]           # es. -5%, -10%, -15%
L_values = [1, 10, 50, 100]             # leve differenti
p_investment_values = [0.05, 0.1, 0.2, 1]  # percentuali diverse da investire

results = []

# Calcoliamo il numero totale di combinazioni
total_combinations = len(TP_values) * len(SL_values) * len(L_values) * len(p_investment_values)
counter = 0

# Iteriamo su tutte le combinazioni
for TP in TP_values:
    for SL in SL_values:
        for L in L_values:
            for p_inv in p_investment_values:
                counter += 1
                print(f"Testando combinazione {counter}/{total_combinations}: TP={TP}, SL={SL}, L={L}, p_investment={p_inv}")
                df_sim = simulate_trading(btc_prices, TP, SL, L, p_inv, initial_liquidity, decide_action)
                # Prendiamo il valore totale finale al termine della simulazione
                final_value = df_sim.iloc[-1]['total_value']
                # Calcoliamo il fattore di rendimento (final_value / liquidità iniziale)
                factor = final_value / initial_liquidity
                print(f"Combinazione {counter}/{total_combinations} terminata con fattore di rendimento: {factor:.4f}\n")
                
                results.append({
                    'TP': TP,
                    'SL': SL,
                    'L': L,
                    'p_investment': p_inv,
                    'final_value': final_value,
                    'factor': factor
                })

# Creiamo un DataFrame con i risultati e lo ordiniamo dal migliore al peggiore
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by='factor', ascending=False)

# Visualizza i risultati della grid search
df_results.reset_index(drop=True, inplace=True)
df_results

Testando combinazione 1/144: TP=0.1, SL=0.05, L=1, p_investment=0.05
Combinazione 1/144 terminata con fattore di rendimento: 1.0453

Testando combinazione 2/144: TP=0.1, SL=0.05, L=1, p_investment=0.1
Combinazione 2/144 terminata con fattore di rendimento: 1.0917

Testando combinazione 3/144: TP=0.1, SL=0.05, L=1, p_investment=0.2
Combinazione 3/144 terminata con fattore di rendimento: 1.1882

Testando combinazione 4/144: TP=0.1, SL=0.05, L=1, p_investment=1
Combinazione 4/144 terminata con fattore di rendimento: 2.1017

Testando combinazione 5/144: TP=0.1, SL=0.05, L=10, p_investment=0.05
Combinazione 5/144 terminata con fattore di rendimento: 1.5043

Testando combinazione 6/144: TP=0.1, SL=0.05, L=10, p_investment=0.1
Combinazione 6/144 terminata con fattore di rendimento: 2.1017

Testando combinazione 7/144: TP=0.1, SL=0.05, L=10, p_investment=0.2
Combinazione 7/144 terminata con fattore di rendimento: 3.3327

Testando combinazione 8/144: TP=0.1, SL=0.05, L=10, p_investment=1
Combin

In [ ]:
print("Risultati della Grid Search:")
df_results